In [34]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

path = 'C:\\Users\\ashri\\Machine Learning Practice\\ML\\19_bagging\\Exercise\\heart.csv'
df = pd.read_csv(path)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


##### 1st we will remove outliers using a ZScore. Anything >3 and anything <-3
##### 2nd we will create two dataframes, one using label encoding and the other using one hot encoding
##### 3rd we will aply scaling
##### 4th we will build a model using svc then one based on a bagging model using svc
##### 5th we will build a model using decision trees and then bagging decision trees.
##### 6th we weill figure out which one works best and where bagging makes the most sense to use and why

In [14]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [15]:
#So let's compute the z-score for RestingBP, Cholesterol, and MaxHR
resting_bp_mean = df['RestingBP'].mean()
cholesterol_mean = df['Cholesterol'].mean()
max_hr_mean = df['MaxHR'].mean()

#Now let's compute the std dev for each.
resting_bp_std = df['RestingBP'].std()
cholesterol_std = df['Cholesterol'].std()
max_hr_std = df['MaxHR'].std()

#Now let's compute the z-score
df['resting_bp_zscore'] = df['RestingBP'].apply(lambda x: ((x-resting_bp_mean)/resting_bp_std))
df['cholesterol_zscore'] = df['Cholesterol'].apply(lambda x: ((x-cholesterol_mean)/cholesterol_std))
df['max_hr_zscore'] = df['MaxHR'].apply(lambda x: ((x-max_hr_mean)/max_hr_std))

In [16]:
#Now, we have the z-scores for the three numerical columns, now we need to remove all Zscores >3 and Zscores <-3
df = df.drop(df[df.resting_bp_zscore < -3].index, axis = 0)
df = df.drop(df[df.resting_bp_zscore > 3].index, axis = 0)
df = df.drop(df[df.cholesterol_zscore < -3].index, axis = 0)
df = df.drop(df[df.cholesterol_zscore > 3].index, axis = 0)
df = df.drop(df[df.max_hr_zscore < -3].index, axis = 0)
df = df.drop(df[df.max_hr_zscore > 3].index, axis = 0)

In [17]:
df.drop(['resting_bp_zscore', 'cholesterol_zscore', 'max_hr_zscore'], axis = 'columns', inplace = True)

In [24]:
#Making the label encoded and one hot encoded dataframes.
df_le = df.copy()
df_ohe = df.copy()

needs_encoding = ['Sex','ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

le = LabelEncoder()

for index in needs_encoding:
    df_le[index + '_le'] = le.fit_transform(df_le[index])
    dummies = pd.get_dummies(df[index], drop_first=True)
    df_ohe = pd.concat([df_ohe, dummies], axis = 'columns')
    

In [25]:
df_le.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_le,ChestPainType_le,RestingECG_le,ExerciseAngina_le,ST_Slope_le
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,1,1,1,0,2
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,0,2,1,0,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,1,1,2,0,2
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0,0,1,1,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,1,2,1,0,2


In [26]:
df_ohe.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,...,HeartDisease,M,ATA,NAP,TA,Normal,ST,Y,Flat,Up
0,40,M,ATA,140,289,0,Normal,172,N,0.0,...,0,1,1,0,0,1,0,0,0,1
1,49,F,NAP,160,180,0,Normal,156,N,1.0,...,1,0,0,1,0,1,0,0,1,0
2,37,M,ATA,130,283,0,ST,98,N,0.0,...,0,1,1,0,0,0,1,0,0,1
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,...,1,0,0,0,0,1,0,1,1,0
4,54,M,NAP,150,195,0,Normal,122,N,0.0,...,0,1,0,1,0,1,0,0,0,1


In [28]:
df_ohe.drop(['Sex','ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], axis = 'columns', inplace = True)
df_le.drop(['Sex','ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], axis = 'columns', inplace = True)

In [29]:
df_le.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_le,ChestPainType_le,RestingECG_le,ExerciseAngina_le,ST_Slope_le
0,40,140,289,0,172,0.0,0,1,1,1,0,2
1,49,160,180,0,156,1.0,1,0,2,1,0,1
2,37,130,283,0,98,0.0,0,1,1,2,0,2
3,48,138,214,0,108,1.5,1,0,0,1,1,1
4,54,150,195,0,122,0.0,0,1,2,1,0,2


In [30]:
df_ohe.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,M,ATA,NAP,TA,Normal,ST,Y,Flat,Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


In [31]:
## Now we need to apply scaling.
needs_scaling = ['Age', 'RestingBP','Cholesterol', 'MaxHR', 'Oldpeak']

scaler = StandardScaler()

for index in needs_scaling:
    df_le[index] = scaler.fit_transform(df_le[[index]])
    df_ohe[index] = scaler.fit_transform(df_ohe[[index]])

In [32]:
df_le.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_le,ChestPainType_le,RestingECG_le,ExerciseAngina_le,ST_Slope_le
0,-1.432687,0.460750,0.848846,0,1.389406,-0.83126,0,1,1,1,0,2
1,-0.477875,1.624344,-0.169349,0,0.756186,0.10838,1,0,2,1,0,1
2,-1.750958,-0.121047,0.792799,0,-1.539237,-0.83126,0,1,1,2,0,2
3,-0.583965,0.344390,0.148253,0,-1.143474,0.57820,1,0,0,1,1,1
4,0.052577,1.042547,-0.029230,0,-0.589407,-0.83126,0,1,2,1,0,2


In [33]:
df_ohe.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,M,ATA,NAP,TA,Normal,ST,Y,Flat,Up
0,-1.432687,0.460750,0.848846,0,1.389406,-0.83126,0,1,1,0,0,1,0,0,0,1
1,-0.477875,1.624344,-0.169349,0,0.756186,0.10838,1,0,0,1,0,1,0,0,1,0
2,-1.750958,-0.121047,0.792799,0,-1.539237,-0.83126,0,1,1,0,0,0,1,0,0,1
3,-0.583965,0.344390,0.148253,0,-1.143474,0.57820,1,0,0,0,0,1,0,1,1,0
4,0.052577,1.042547,-0.029230,0,-0.589407,-0.83126,0,1,0,1,0,1,0,0,0,1


In [40]:
#Now let's make a SVC model for the data

x_le = df_le.drop('HeartDisease', axis = 'columns')
x_ohe = df_ohe.drop('HeartDisease', axis = 'columns')
y = df_le.HeartDisease

x_le_train, x_le_test, y_le_train, y_le_test = train_test_split(x_le, y)
x_ohe_train, x_ohe_test, y_ohe_train, y_ohe_test = train_test_split(x_ohe, y)

In [41]:
svc_le_model = SVC()
svc_ohe_model = SVC()

svc_le_model.fit(x_le_train, y_le_train)
svc_ohe_model.fit(x_ohe_train, y_ohe_train)

SVC()

In [42]:
svc_le_model.score(x_le_test, y_le_test)

0.8370044052863436

In [43]:
svc_ohe_model.score(x_ohe_test, y_ohe_test)

0.8766519823788547

In [51]:
bag_le_model = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=20,
    max_samples=0.8,
    oob_score=True,
    random_state=3
)

bag_ohe_model = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=20,
    max_samples=0.8,
    oob_score=True,
    random_state=3
)

bag_le_model.fit(x_le_train, y_le_train)
bag_ohe_model.fit(x_ohe_train, y_ohe_train)

BaggingClassifier(base_estimator=SVC(), max_samples=0.8, n_estimators=20,
                  oob_score=True, random_state=3)

In [52]:
bag_le_model.score(x_le_test,y_le_test)

0.8370044052863436

In [53]:
bag_ohe_model.score(x_ohe_test,y_ohe_test)

0.8854625550660793

In [54]:
#Now for a decision tree model

dt_le_model = DecisionTreeClassifier()
dt_ohe_model = DecisionTreeClassifier()

dt_le_model.fit(x_le_train, y_le_train)
dt_ohe_model.fit(x_ohe_train,y_ohe_train)

DecisionTreeClassifier()

In [55]:
dt_le_model.score(x_le_test,y_le_test)

0.73568281938326

In [56]:
dt_ohe_model.score(x_ohe_test, y_ohe_test)

0.801762114537445

In [60]:
bag_le_model_dt = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=20,
    max_samples=0.8,
    oob_score=True,
    random_state=3
)

bag_ohe_model_dt = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=20,
    max_samples=0.8,
    oob_score=True,
    random_state=3
)

bag_le_model_dt.fit(x_le_train, y_le_train)
bag_ohe_model_dt.fit(x_ohe_train, y_ohe_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=20, oob_score=True, random_state=3)

In [61]:
bag_le_model_dt.score(x_le_test, y_le_test)

0.8370044052863436

In [63]:
bag_ohe_model_dt.score(x_ohe_test, y_ohe_test)

0.8370044052863436